回想一下softmax函数$\hat{y}_j = \frac{\exp(o_j)}{\sum_k \exp(o_k)}$，其中$\hat{y}_j$是预测的概率分布。$o_j$是未规范化的预测$\mathbf{o}$的第$j$个元素。如果$o_k$中的一些数值非常大，那么$\exp(o_k)$可能大于数据类型容许的最大数字，即上溢（overflow）。这将使分母或分子变为inf（无穷大），最后得到的$\hat{y}_j$是0、inf或nan（不是数字）。在这些情况下，我们无法得到一个明确定义的交叉熵值。

解决上述问题的一个技巧是：在继续softmax运算之前，先从所有$o_k$中减去$\max(o_k)$。这里可以看到每个$o_k$按常数进行的移动不会改变softmax的返回值：
$$
\begin{align*}
\hat{y}_j &= \frac{exp(o_j-\max(o_k))exp(\max(o_k))}{\sum_k \exp(o_k-\max(o_k))exp(\max(o_k))} \\
&= \frac{exp(o_j-\max(o_k))}{\sum_k \exp(o_k-\max(o_k))}
\end{align*}
$$


在执行减法和规范化步骤之后，可能有些$o_j - max(o_k)$具有较大的复制。由于精度受限，$exp(o_j - max(o_k))$将有接近零的值，即下溢（underflow）。这些值可能会四舍五入为零，使$\hat{y}_j$为零，并且使$\log(\hat{y}_j)$的值为$-inf$。反向传播几步后，我们会发现可能面对满屏的$nan$。

尽管我们要计算指数函数，但我们最终在计算交叉熵损失时会取它们的对数。通过将softmax和交叉熵结合在一起，可以避免反向传播过程中可能会困扰我们的数值稳定性问题。如下面的等式所示，我们可以避免计算$\exp(o_j - \max(o_k))$，而可以直接使用$o_j - \max(o_k)$，因为$\log(\exp(\cdot))$等价于$\cdot$。
$$\begin{align*}
\log(\hat y_j) &= \log\left(\frac{\exp(o_j - \max(o_k))}{\sum_k \exp(o_k-\max(o_k))}\right) \\
&= \log(\exp(o_j - \max(o_k))) - \log\left(\sum_k \exp(o_k-\max(o_k))\right) \\
&= o_j - \max(o_k) - \log\left(\sum_k \exp(o_k-\max(o_k))\right)
\end{align*}
$$

我们希望保留传统的softmax函数，以备需要评估通过模型输出的概率。但是，我们没有将softmax概率传递到损失函数中，而是在交叉熵损失函数中传递未规范化的预测，并同时计算softmax及其对数，这是一种类似于“LogSumExp技巧”的聪明方式。